In [4]:
"""
{
  [assessment_name]: {
    "description": [description text],
    "url": [reproschema_url],
    "data elements": {
       [element_name]: {
          'description': [description text],
          'datatype': <type>,
          'choices': [
             [choice name],
             [another choice name],
           ],
           'termURL': [reproschema_url]
       }
     } 
   }
}
"""

"""
Ex now:
{
    "adhd_session_id": {
        "description": "Unique identifier for the ADHD session."
    }
}
"""
#dict of non-items only that match
#list of keys that don't correspond to any reproschema files
#
# def
# for each phenotype jsons
#   open as dict
#   for each key
#       recursively search reproschema for item files that match
#       if found at least one:
#           if at least one type item
#               then populate data
#           else
#              add to list of non-items only that match 
#       else:
#           add to list of keys that don't correspond to any reproschema files
#
# def populate data
#           

'\nEx:\n{\n    "adhd_session_id": {\n        "description": "Unique identifier for the ADHD session."\n    }\n'

In [16]:
import os
import json

def search_string_in_json_files(directory, search_string):
    matching_files = []  # List to store paths of matching JSON files

    # Walk through each directory and file in the given directory
    for root, dirs, files in os.walk(directory):
        for phenotype_file_name in files:
            file_path = os.path.join(root, phenotype_file_name)
            try:
                # Attempt to open and load the file as JSON
                with open(file_path, 'r', encoding='utf-8') as file:
                    data = json.load(file)  # Load JSON data
                    # Check if the search string is present in the JSON content
                    if search_string in str(data):
                        matching_files.append(file_path)
            except Exception:
                # Skip files that cannot be read or loaded as JSON
                continue
    
    return matching_files  # Return the list of matching file paths
# search_string = "adhd_session_id"
# directory = "/Users/isaacbevers/sensein/reproschema-wrapper/b2ai-redcap2rs/activities/q_mood_adhd_adult/items"
# search_string_in_json_files(directory, search_string)


In [71]:
def populate_data_element(phenotype_file_dict, key, item_file_path, phenotype_file_name):
    with open(item_file_path, 'r', encoding='utf-8') as file:
        reproschema_item = json.load(file)
    if not phenotype_file_dict[key] or not isinstance(phenotype_file_dict[key], dict):
        phenotype_file_dict[key] = {}
    phenotype_file_dict[key]["question"] = reproschema_item["question"]
    phenotype_file_dict[key]["datatype"] = reproschema_item["responseOptions"]["valueType"]
    if 'choices' in reproschema_item["responseOptions"]:
        phenotype_file_dict[key]["choices"] = reproschema_item["responseOptions"]['choices']
    elif "choices" in reproschema_item["responseOptions"]:
        phenotype_file_dict[key]["choices"] = reproschema_item["responseOptions"]["choices"]
    else:
        phenotype_file_dict[key]["choices"] = None
    
    #TODO add URL

    # phenotype_file_dict[phenotype_file_name][""] = # question field

        #       'description': [description text],
        #   'datatype': <type>,
        #   'choices': [
        #      [choice name],
        #      [another choice name],
        #    ],
        #    'termURL': [reproschema_url]

In [72]:
matching_non_item_files = {}
multiple_item_files = []
non_matching = []

# Specify the phenotype_dir containing the .json files
phenotype_dir = "/Users/isaacbevers/sensein/b2ai-wrapper/b2aiprep/src/b2aiprep/prepare/resources/b2ai-data-bids-like-template/phenotype"
b2ai_redcap2rs_activities_dir = "/Users/isaacbevers/sensein/reproschema-wrapper/b2ai-redcap2rs/activities"
# Dictionary to store the loaded JSON data

def main():
    # Loop through each file in the phenotype_dir
    for phenotype_file_name in os.listdir(phenotype_dir):
        # Check if the file ends with .json and is not "<measurement_tool_name>.json"
        if phenotype_file_name.endswith(".json") and phenotype_file_name != "<measurement_tool_name>.json":
            file_path = os.path.join(phenotype_dir, phenotype_file_name)

            # Open and load the JSON file
            with open(file_path, 'r', encoding='utf-8') as file:
                phenotype_file_dict = json.load(file)
        
            phenotype_file_dict["data_elements"] = {}
            for key in phenotype_file_dict:
                file_paths = search_string_in_json_files(b2ai_redcap2rs_activities_dir, key)
                if file_paths:
                    item_file_paths = [path for path in file_paths if "item" in path]
                    if item_file_paths and len(item_file_paths) == 1:
                        populate_data_element(phenotype_file_dict, key, item_file_paths[0], phenotype_file_name)
                    elif item_file_paths and len(item_file_paths) > 1:
                        multiple_item_files.append(item_file_paths)
                    else:
                        matching_non_item_files[key] = file_paths
                else:
                    non_matching.append(key)

            # TODO
            # phenotype_file_dict[phenotype_file_name]["url"] = 
            #phenotype_file_dict[phenotype_file_name][data elements] = 
            # if phenotype_file_name not in phenotype_file_dict:
            #     phenotype_file_dict = {phenotype_file_name: RS ASSESSMENT NAME}
            with open(file_path, 'w', encoding='utf-8') as file:
                json.dump(phenotype_file_dict, file, ensure_ascii=False, indent=4)

            # save data

#       if found at least one:
#           if at least one type item
#               then populate data
#           else
#              add to list of non-items only that match 
#       else:
#           add to list of keys that don't correspond to any reproschema files
        
main()


{'id': 'vhi_session_id', 'category': 'reproschema:Item', 'preamble': {'en': 'Questionnaire - Metadata'}, 'prefLabel': {'en': 'vhi_session_id'}, 'question': {'en': 'Session ID'}, 'responseOptions': {'valueType': ['xsd:string']}, 'ui': {'inputType': 'text'}, '@context': 'https://raw.githubusercontent.com/ReproNim/reproschema/main/releases/1.0.0/reproschema'}
{'id': 'vhi_duration', 'category': 'reproschema:Item', 'prefLabel': {'en': 'vhi_duration'}, 'question': {'en': 'Questionnaire Duration (seconds)'}, 'responseOptions': {'minValue': 0.0, 'valueType': ['xsd:decimal']}, 'ui': {'inputType': 'float'}, '@context': 'https://raw.githubusercontent.com/ReproNim/reproschema/main/releases/1.0.0/reproschema'}
{'id': 'diagnosis_vfp_gsd', 'category': 'reproschema:Item', 'preamble': {'en': '1. Gold Standard Diagnosis'}, 'prefLabel': {'en': 'diagnosis_vfp_gsd'}, 'question': {'en': 'Does the patient have VFP confirmed by laryngoscopy/stroboscopy?'}, 'responseOptions': {'choices': [{'name': {'en': 'No '